In [ ]:
import os
from datetime import datetime, timedelta
import glob

from cartopy.crs import NorthPolarStereo
from cartopy.feature import LAND, COASTLINE
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
from pysida.lib import DAY_SECONDS

In [ ]:
def decorate_map(ax, map_extent, crs, title):
    ax.add_feature(LAND)
    ax.add_feature(COASTLINE)
    ax.set_extent(map_extent, crs=crs)
    ax.set_title(title)
    ax.plot(0, 0, 'bo')
    ax.text(30000, -30000, 'North \nPole')

srs_dst = NorthPolarStereo(central_longitude=-45, true_scale_latitude=60)
map_extent = [-2300000, 300000, -1000000, 2100000]
titles = ['RGPS', 'BBM']


In [ ]:
# EXP 01
param_names = ['compression_factor', 'C_lab', 'nu0', 'tan_phi']
imask = '../run_experiment/tru_cfg_01/sa10free_mat*cfg'

param_vals = {}
tru_cfg_files = sorted(glob.glob(imask))
for tru_cfg_file in tru_cfg_files:
    exp_num = tru_cfg_file.split('/')[-1].split('.')[0].split('_')[1].replace('mat','')
    param_vals[exp_num] = {}
    with open(tru_cfg_file) as f:
        lines = f.readlines()

    for line in lines:
        for param_name in param_names:
            if param_name in line:
                param_vals[exp_num][param_name] = float(line.strip().split('=')[1])


In [ ]:
param_names = ['compression_factor', 'C_lab']
param_val_arr = [[param_vals[exp][param_name] for param_name in param_names] for exp in param_vals]
param_val_arr = np.array(param_val_arr)
param_val_std = np.std(param_val_arr, axis=0)#[None]
param_val_avg = np.mean(param_val_arr, axis=0)#[None]
param_val_nrm = (param_val_arr - param_val_avg) / param_val_std

best_vals_arrs = [
    np.array([   500, 1.6e6]), # lowest compression_factor
    np.array([ 20000, 1.6e6]), # highest compression_factor
    np.array([ 10000, 0.5e6]), # lowest C_lab
    np.array([ 10000, 3.0e6]), # highest C_lab
]


i = 1
plt.plot(param_val_arr.T[i], param_val_arr.T[0], 'k.')

best_param_is = []
color_index = np.arange(51)
symbols = ['*', 'x', 'P', 'v']


for best_vals_arr_idx, best_vals_arr in enumerate(best_vals_arrs):
    best_vals_nrm = (best_vals_arr - param_val_avg) / param_val_std

    best_param_i = np.argmin(np.sum((param_val_nrm - best_vals_nrm)**2, axis=1))
    best_param_is.append(best_param_i)
    print(best_param_i, best_vals_arr, param_val_arr[best_param_i])

    exp_nums = np.array([int(exp) for exp in param_vals])

    plt.plot(param_val_arr[best_param_i, i], param_val_arr[best_param_i, 0], 'g'+symbols[best_vals_arr_idx], ms=12, alpha=0.5)
    plt.xlabel(param_names[i])
plt.ylabel(param_names[0])
plt.show()


In [ ]:
idir = './music_matrix/cfg01_m20'
best_param_is_4use = list(best_param_is)
pfiles = [f'{idir}/mat{best_param_i:02}_pairs.npz' for best_param_i in best_param_is_4use]
sources = ['neXtSIM'] * len(pfiles)

pfiles += ['./rgps_csv/w07_may_pairs.npz']
sources += ['RGPS']
force = True

for pfile, src in zip(pfiles, sources):
    dfile = pfile.replace('_pairs.npz', '_defor.npz')
    pairs = np.load(pfile, allow_pickle=True)['pairs']
    defor = np.load(dfile, allow_pickle=True)['defor']

    date0 = datetime(2007,1,1)
    period_len = timedelta(4)
    time_step = 3

    e_name = 'Total deformation'

    for day in range(0, 48, time_step):
        date1 = date0 + timedelta(day)
        bfile = pfile.rstrip('_pairs.npz')
        ofile = f'../tuning_paper_figures/{bfile}_tot_{date1.strftime("%Y%m%d")}.png'
        if os.path.exists(ofile) and not force:
            continue

        date_str = date1.strftime('%Y-%m-%d')
        print(date1)
        fig, axs = plt.subplots(1,1,figsize=(10,10), subplot_kw={'projection': srs_dst})
        for p, d in zip(pairs, defor):
            if p is None or d is None:
                continue
            if date1 <= p.d1 < (date1 + period_len):
                e = np.hypot(d.e1, d.e2) * DAY_SECONDS
                trp = axs.tripcolor(p.x1, p.y1, e, triangles=p.t, vmin=0, vmax=0.1, cmap='plasma_r', mask=~p.g)
        decorate_map(axs, map_extent, srs_dst, src)
        plt.tight_layout()

        bgax = inset_axes(axs, '95%', '10%', loc='lower center')
        bgax.set_facecolor((1,1,1,0.9))
        bgax.spines['top'].set_visible(False)
        bgax.spines['right'].set_visible(False)
        bgax.spines['bottom'].set_visible(False)
        bgax.spines['left'].set_visible(False)
        bgax.tick_params(colors=(0,0,0,0), )
        cbar_ax = inset_axes(bgax, '95%', '20%', loc='upper center')
        cbar = fig.colorbar(trp, cax=cbar_ax, orientation='horizontal')
        cbar.set_label(f'{e_name}, {date_str}, 1/day', fontsize=14)

        print(ofile)
        plt.savefig(ofile, dpi=150, bbox_inches='tight', pad_inches=0)
        plt.close()


In [ ]:
#13 [5.0e+02 1.6e+06] [6.000e+02 1.725e+06]
#37 [  20000. 1600000.] [  19000. 1575000.]
#2 [ 10000. 500000.] [  8200. 525000.]
#26 [  10000. 3000000.] [   9800. 2875000.]
idir = './music_matrix/cfg01_m20'

n_p_hi = sorted(glob.glob(f'{idir}/mat13_tot*.png'))
n_p_lo = sorted(glob.glob(f'{idir}/mat37_tot*.png'))
n_c_hi = sorted(glob.glob(f'{idir}/mat02_tot*.png'))
n_c_lo = sorted(glob.glob(f'{idir}/mat26_tot*.png'))
r_s = sorted(glob.glob('./rgps_csv/*tot*.png'))

len(n_p_hi), len(n_p_lo), len(n_c_hi), len(n_c_lo), len(r_s),

In [ ]:
texts = ['RGPS', 'low P', 'high P', 'low C', 'high C']
for r, phi, plo, chi, clo in zip(r_s, n_p_hi, n_p_lo, n_c_hi, n_c_lo):
    fig, axs = plt.subplots(1,5, figsize=(20,5))
    for i, (t, f) in enumerate(zip(texts, [r, plo, phi, clo, chi])):
        a = plt.imread(f)
        axs[i].imshow(a[100:-100])
        axs[i].text(230, 70, t, fontsize=16)
    for ax in axs.flat:
        ax.axis('off')
    date = r.split('.')[-2].split('_')[-1]
    axs[0].text(10, 750, date, fontsize=16, rotation=90)
    plt.tight_layout()
    #plt.show()
    ofile = f'{idir}/row_tot_{date}.png'
    plt.savefig(ofile, dpi=150, bbox_inches='tight', pad_inches=0)
    plt.close()
    print(ofile)
